In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px

In [2]:
df = pd.read_csv(r"datasets/clean_data.csv")

In [3]:
v = df["vote_count"]
C = df["vote_average"].mean()
R = df["vote_average"]

In [4]:
# m = 1900
m = int(df["vote_count"].quantile(0.9))

In [5]:
def weighted_avg_rating(x, m=m, C=C):
    v = x["vote_count"]
    R = x["vote_average"]

    return ((R * v) + (C * m)) / (v + m)

In [ ]:
data = df[df["vote_count"] >= m].copy()
data.head()

In [ ]:
data["weighted_avg"] = data.apply(weighted_avg_rating, axis=1)
data.head()

In [ ]:
df_sorted_ranking = data.sort_values(by="weighted_avg", ascending=False)
df_sorted_ranking.head()

In [ ]:
popularity = data.sort_values(by="popularity", ascending=False)
popularity.head()

In [20]:
scaling = MinMaxScaler()

In [21]:
popularity[["weighted_avg", "popularity"]].head()

,weighted_avg,popularity
546,6.311720,875.581305
95,7.809533,724.247784
788,7.212687,514.569956
94,7.613058,481.098624
127,7.019246,434.278564


In [27]:
df_scaled = scaling.fit_transform(popularity[["weighted_avg", "popularity"]])
df_scaled[:5]

array([[0.39814088, 1.        ],
       [0.91397055, 0.82679441],
       [0.70842373, 0.58681208],
       [0.84630695, 0.54850318],
       [0.64180504, 0.49491627]])

In [13]:
popularity[["weighted_avg_scaled", "popularity_scaled"]] = df_scaled

In [29]:
popularity["score_mix"] = (
    popularity["weighted_avg_scaled"] * 0.5 + popularity["popularity_scaled"] * 0.50
)
popularity["score_mix"].head()

546    0.699070
95     0.870382
788    0.647618
94     0.697405
127    0.568361
Name: score_mix, dtype: float64

In [32]:
popularity_sorted = popularity.sort_values(by="score_mix", ascending=False)
popularity_sorted.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,release_date,...,spoken_languages,tagline,vote_average,vote_count,cast,crew,weighted_avg,weighted_avg_scaled,popularity_scaled,score_mix
95,165000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 18, ""...",157336,"[{""id"": 83, ""name"": ""saving the world""}, {""id""...",en,Interstellar,Interstellar chronicles the adventures of a gr...,724.247784,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...",2014-11-05,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Mankind was born on Earth. It was never meant ...,8.1,10867,"[{""cast_id"": 9, ""character"": ""Joseph Cooper"", ...","[{""credit_id"": ""52fe4bbf9251416c910e4801"", ""de...",7.809533,0.913971,0.826794,0.870382
546,74000000,"[{""id"": 10751, ""name"": ""Family""}, {""id"": 16, ""...",211672,"[{""id"": 3487, ""name"": ""assistant""}, {""id"": 179...",en,Minions,"Minions Stuart, Kevin and Bob are recruited by...",875.581305,"[{""name"": ""Universal Pictures"", ""id"": 33}, {""n...",2015-06-17,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","Before Gru, they had a history of bad bosses",6.4,4571,"[{""cast_id"": 22, ""character"": ""Scarlet Overkil...","[{""credit_id"": ""5431b2b10e0a2656e20026c7"", ""de...",6.311720,0.398141,1.000000,0.699070
94,170000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 878, ""na...",118340,"[{""id"": 8828, ""name"": ""marvel comic""}, {""id"": ...",en,Guardians of the Galaxy,"Light years from Earth, 26 years after being a...",481.098624,"[{""name"": ""Marvel Studios"", ""id"": 420}, {""name...",2014-07-30,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",All heroes start somewhere.,7.9,9742,"[{""cast_id"": 1, ""character"": ""Peter Quill / St...","[{""credit_id"": ""538ce329c3a3687155003358"", ""de...",7.613058,0.846307,0.548503,0.697405
788,58000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",293660,"[{""id"": 2095, ""name"": ""anti hero""}, {""id"": 307...",en,Deadpool,Deadpool tells the origin story of former Spec...,514.569956,"[{""name"": ""Twentieth Century Fox Film Corporat...",2016-02-09,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Witness the beginning of a happy ending,7.4,10995,"[{""cast_id"": 99, ""character"": ""Wade Wilson / D...","[{""credit_id"": ""56c986b2925141172f0068b6"", ""de...",7.212687,0.708424,0.586812,0.647618
65,185000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...",155,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight,Batman raises the stakes in his war on crime. ...,187.322927,"[{""name"": ""DC Comics"", ""id"": 429}, {""name"": ""L...",2008-07-16,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Why So Serious?,8.2,12002,"[{""cast_id"": 35, ""character"": ""Bruce Wayne"", ""...","[{""credit_id"": ""55a0eb4a925141296b0010f8"", ""de...",7.920073,0.952039,0.212268,0.582154


In [16]:
px.bar(
    popularity_sorted.head(10),
    x="original_title",
    y="score_mix",
    labels={
        "original_title": "Movie Title",
        "score_mix": "Weighted Popularity Score",
    },
    title="Top 10 Movies by Weighted Popularity",
    width=1200,
    height=500,
    color="original_title",
)